In [183]:
import numpy as np
from collections import Counter
from math import sqrt, log
import time #used to give some statistics on the execution time
import random

In [184]:
NUM_COLUMNS = 7
COLUMN_HEIGHT = 6
FOUR = 4

# Board can be initiatilized with `board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)`
# Notez Bien: Connect 4 "columns" are actually NumPy "rows"

In [185]:
def valid_moves(board):
    """Returns columns where a disc may be played"""
    return [n for n in range(NUM_COLUMNS) if board[n, COLUMN_HEIGHT - 1] == 0]


def play(board, column, player):
    """Updates `board` as `player` drops a disc in `column`"""
    (index,) = next((i for i, v in np.ndenumerate(board[column]) if v == 0))
    board[column, index] = player


def take_back(board, column):
    """Updates `board` removing top disc from `column`"""
    (index,) = [i for i, v in np.ndenumerate(board[column]) if v != 0][-1]
    board[column, index] = 0


def four_in_a_row(board, player):
    """Checks if `player` has a 4-piece line"""
    return (
        any(
            all(board[c, r] == player)
            for c in range(NUM_COLUMNS)
            for r in (list(range(n, n + FOUR)) for n in range(COLUMN_HEIGHT - FOUR + 1))
        )
        or any(
            all(board[c, r] == player)
            for r in range(COLUMN_HEIGHT)
            for c in (list(range(n, n + FOUR)) for n in range(NUM_COLUMNS - FOUR + 1))
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co, co + FOUR))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
        or any(
            np.all(board[diag] == player)
            for diag in (
                (range(ro, ro + FOUR), range(co + FOUR - 1, co - 1, -1))
                for ro in range(0, NUM_COLUMNS - FOUR + 1)
                for co in range(0, COLUMN_HEIGHT - FOUR + 1)
            )
        )
    )
def turn(board):
  if np.sum(board) <= 0:
    return 1
  else:
    return -1
  
def eval_terminal(board):
  if four_in_a_row(board, 1):
    return 1
  elif four_in_a_row(board, -1):
    return -1
  else:
    return 0
#checks if the last move is a winning move -> faster than using four_in_a_row on the whole board
def winning_move(board, col, player):
  # x, y pos of the play in "board"
  x = col
  y = np.sum(np.where(board[x, :] != 0, 1, 0)) - 1
  # vertical
  if y >= FOUR-1 and np.sum(board[x, y-FOUR+1:y+1]) == FOUR*player:
    return True
  # horizontal
  elif any([np.sum(board[x-i:x-i+FOUR, y]) == FOUR*player for i in range(FOUR) if x-i >= 0 and x-i+FOUR <= NUM_COLUMNS]):
    return True
  # diagonal
  elif any([np.sum(board[x-i:x-i+FOUR, y-i:y-i+FOUR].diagonal()) == FOUR*player for i in range(FOUR) if x-i >= 0 and x-i+FOUR <= NUM_COLUMNS and y-i >=0 and y-i+FOUR <= COLUMN_HEIGHT]):
    return True
  # inverse diagonal
  elif any([np.sum(np.fliplr(board[x+i+1-FOUR:x+i+1, y-i:y-i+FOUR]).diagonal()) == FOUR*player for i in range(FOUR) if x+i+1-FOUR >= 0 and x+i+1 <= NUM_COLUMNS and y-i >=0 and y-i+FOUR <= COLUMN_HEIGHT]):
    return True
  return False


In [186]:
def _mc(board, player):
    p = -player
    while valid_moves(board):
        p = -p
        c = np.random.choice(valid_moves(board))
        play(board, c, p)
        if winning_move(board, c, p):
            return p


def montecarlo(board, player, samples):
    montecarlo_samples = samples
    cnt = Counter(_mc(np.copy(board), player) for _ in range(montecarlo_samples))
    return (cnt[1] - cnt[-1]) / montecarlo_samples


def eval_board(board, player, samples):
    if four_in_a_row(board, 1):
        # Alice won
        return 1
    elif four_in_a_row(board, -1):
        # Bob won
        return -1
    else:
        # Not terminal, let's simulate...
        return montecarlo(board, player, samples)

In [187]:
# With the default configuration it is very smart and it takes an average of 6 seconds to compute the move, based on the game state
# Using depth = 2 grealty increases the speed (~1 second), with a not too big loss in performance
# Changing the samples of the MC simulation increases the accuracy of the evaluation of a non-terminal state, increasing the averall performance (and decreasing speed)
def _minmax(board, player, depth=3, samples=50, alpha=-1, beta=1):
  moves = valid_moves(board)
  val = eval_terminal(board)
  if val == 1 or val == -1 or not moves:
      return None, val
  if depth == 0:
    return None, eval_board(board, player, samples)

  evaluations = []
  if player == 1:
    for move in sorted(moves, key=lambda x: abs(int(NUM_COLUMNS/2)-x)):
      newboard = board.copy()
      play(newboard, move, player)
      _, val = _minmax(newboard, -player, depth-1, samples, alpha, beta)
      evaluations.append((move, val))
      alpha = max(alpha, val)
      if beta <= alpha: 
        break  
    return max(evaluations, key = lambda x: x[1])

  else:
    for move in sorted(moves, key=lambda x: abs(int(NUM_COLUMNS/2)-x)):
      newboard = board.copy()
      play(newboard, move, player)
      _, val = _minmax(newboard, -player, depth-1, samples, alpha, beta)
      evaluations.append((move, val))
      beta = min(beta, val)
      if beta <= alpha: 
        break    
    return min(evaluations, key = lambda x: x[1])

def minmax(board, player, depth=3, samples=50, alpha=-1, beta=1):
  #if there is a winning play, go for it
  wm = getWinningMoves(board, valid_moves(board), player)
  if wm:
    return wm[0]
  if not valid_moves(board):
    return None
  return _minmax(board, player, depth, samples, alpha, beta)[0]

In [188]:
class Node:
  def __init__(self, state, player, play=None, father=None, n=0):
    self.state = state
    self.player = player
    self.father = father
    self.play = play
    self.moves = valid_moves(state)
    # if a player can win, it makes no sense to try other moves because he will always go for the winning move
    wm = getWinningMoves(state, self.moves, player) 
    if wm:
      self.moves = wm
    random.shuffle(self.moves)
    self.children = []
    self.wins = 0
    self.n = n
    self.finalVal = None

def getWinningMoves(board, moves, player):
  winningMoves = []
  for move in moves:
    newboard = board.copy()
    play(newboard, move, player)
    if winning_move(newboard, move, player):
      winningMoves.append(move)
  return winningMoves

#upper confidence bound
def UCB(wins, n, N, c=sqrt(2)):
  if n == 0:
    return inf
  mean = wins / n
  x = c * sqrt(log(N) / n)
  return mean + x

def Simulation(node):
  tmpboard = node.state.copy()
  
  p = -node.player
  while valid_moves(tmpboard):
    p = -p
    c = np.random.choice(valid_moves(tmpboard))
    play(tmpboard, c, p)
    if winning_move(tmpboard, c, p):
      if p == -1:
        p = 0
      return p
  return 0.5
# Using 10000 iterations it takes an average of 5 seconds to compute the output
def MCTS(board, player, iterations=10000):
  val = eval_terminal(board)
  if val:
    return None
  if not valid_moves(board):
    return None
  
  #if there is a winning play, go for it
  wm = getWinningMoves(board, valid_moves(board), player)
  if wm:
    return wm[0]

  root = Node(board, player)

  for _ in range(iterations):
    node = root
    #Selection
    # Stop when there are possible moves to try, when there aren't any childre or when it is a terminal node
    while not node.moves and node.children and node.finalVal is None:
      node = max(node.children, key=lambda x: UCB(x.wins, x.n, node.n))
    newNode = node
    
    #Expansion
    # If the selected node has a final state, no need to compute it again
    if node.finalVal is not None:
      val = node.finalVal
    elif node.moves:
      newboard = node.state.copy()
      play(newboard, node.moves[0], node.player)
      newNode = Node(newboard, father=node, player=-node.player, play=node.moves[0])
      node.children.append(newNode)
      node.moves.pop(0)
      #print(newNode.state)
      #pdb.set_trace()
      # if the move ends the game, set the final state and remove any possible move/children of that node
      if winning_move(newboard, newNode.play, node.player):
        val = node.player
        newNode.moves = []
        newNode.children = []
        if val == 1:
          newNode.finalVal = 1
        elif val == -1:
          newNode.finalVal = 0
          val = 0
      else:
        #Simulation
        val = Simulation(newNode)


    #backpropagation
    while newNode.father != None:
      newNode.n += 1
      newNode.wins += val
      newNode = newNode.father
    newNode.n += 1
    newNode.wins += val
  
  # for child in root.children:
  #   print(child.play, child.wins, child.n)

  if player == 1:
    return max(root.children, key=lambda x: x.wins/x.n).play
  else:
    return min(root.children, key=lambda x: x.wins/x.n).play

In [189]:
# board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)

# Minmax - 1 move

# print(board)
# col = minmax(board, 1)
# print(col, cnt)
# play(board, col, 1)
# print(board)

#Monte Carlo - 1 move

# print(board)
# col = MCTS(board, 1)
# play(board, col, 1)
# print(board, col)

player = 1
time_MCTS = 0
time_minmax = 0
n_minmax = 0
n_MCTS = 0
mc = 0
mm = 0

# Minmax is ways smarter, I tried to optimize MC because minmax with alphabeta pruning and with the new "winning_move" function was already incredibly fast and smart
# Using the final state and the winning moves list in MC greatly increased the performance and also the speed, but still not able to win against minmax if not due to some luck
# Sometimes it seems they make random moves, but they're playing "random" because in that situation the they had no more possibilities of winning
for _ in range(3): # Number of games to play
  board = np.zeros((NUM_COLUMNS, COLUMN_HEIGHT), dtype=np.byte)
  # Monte Carlo vs Minmax
  while eval_terminal(board) != 1 and eval_terminal(board) != -1:
    if player == 1:
      start = time.time()
      move = MCTS(board.copy(), 1, 10000)
      end = time.time()
      time_MCTS += end - start
      n_MCTS += 1
      print("MCTS plays ", move)
    else:
      start = time.time()
      move = minmax(board.copy(), -1)
      end = time.time()
      time_minmax += end - start
      n_minmax += 1
      print("Minmax plays ", move)
    if move is not None:
      play(board, move, player)
    player = -player
    print(board)
  if eval_terminal(board) == 1:
    mc += 1
    print("Montecarlo wins!")
  elif eval_terminal(board) == -1:
    mm += 1
    print("Minmax wins!")
  else:
    print("Draw!")
    mm += 0.5
    mc += 0.5
  print("Minmax - Montecarlo: ", mm, " - ", mc)
  print()
print("Minmax total time: ", time_minmax)
print("Minmax mean time: ", time_minmax/n_minmax)
print("MCTS total time: ", time_MCTS)
print("MCTS mean time: ", time_MCTS/n_MCTS)

MCTS plays  3
[[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [1 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
Minmax plays  3
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
MCTS plays  4
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
Minmax plays  2
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
MCTS plays  5
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 0  0  0  0  0  0]]
Minmax plays  6
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [-1  0  0  0  0  0]
 [ 1 -1  0  0  0  0]
 [ 1  0  0  0  0  0]
 [ 1  0  0  0  0  0]
 [-1  0  0  0  0  0]]
MCTS plays  4
[[ 0  0  0  0  0  0]
 [ 0  0  0  0  0  0]
 [-1  0 